In [2]:
#Finding the difference for each spider mean in each group and plotting it 
#Finding the difference for each spider mean 



In [3]:
# import pandas as pd
# import numpy as np
# from scipy import stats

# # Read data
# col_names = ["Index", "DateD", "DateM", "DateY", "Time", "MonStatus", "Extras", "MonN", "TubeN", "DataType", "Unused", "Light"]
# for i in range(1, 33):
#     col_names.append(f"S-i}")
# file = 'Steatoda A masking 02 pm.txt'
# df = pd.read_csv(file, names=col_names, sep='\s+', header=None)
# df = df.set_index('Index')

# # Preprocess data
# df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S', errors='coerce')
# df = df[df["MonStatus"] == 1]
# month_map = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6}
# df['DateM'] = df['DateM'].str[:3].map(month_map)
# df['DateY'] = df['DateY'].apply(lambda x: int(str(20) + str(x)))
# df['Date'] = pd.to_datetime(dict(year=df['DateY'], month=df['DateM'], day=df['DateD']), errors='coerce')
# df['Time'] = pd.to_datetime(dict(year=df['Date'].dt.year, month=df['Date'].dt.month, day=df['Date'].dt.day, hour=df['Time'].dt.hour, minute=df['Time'].dt.minute, second=df['Time'].dt.second))
# df = df.drop(["DateD", "DateM", "DateY", "Date", "MonStatus", "Extras", "MonN", "TubeN", "DataType", "Unused"], axis=1)
# day_map = {day: idx+1 for idx, day in enumerate(df['Time'].dt.day.unique())}
# df.insert(0, 'Day', df['Time'].dt.day.map(day_map))

# # Function to filter and merge data
# def filter_and_merge(df, threshold=0):
#     additional_columns = df.columns[:3]
#     filtered_dfs = []

#     for day in range(1, 9):  
#         day_df = df[df['Day'] == day] 
#         count_mov = day_df.filter(like="Sp")  
#         x = count_mov.sum(axis=0)  
#         z = x > threshold  
#         columns_to_keep = z[z].index 
        
#         all_columns_to_keep = list(additional_columns) + list(columns_to_keep)
#         filtered_df = day_df[all_columns_to_keep]
#         filtered_dfs.append(filtered_df)

#     merged_df = pd.concat(filtered_dfs)
#     merged_df = merged_df.dropna(axis=1)
    
#     return merged_df

# # Filter and merge data
# merged_df = filter_and_merge(df)

# # Function to calculate paired t-test for each spider group
# def paired_t_test_spiders(df):
#     diff_2_4 = []
#     diff_pulse = []

#     for spider_column in df.columns[3:]:  # Exclude non-numeric columns
#         avg_activities_24hrs_2_4 = df[(df['Day'].isin(range(2, 5)))][spider_column].mean()
#         avg_activities_pulse = df[(df['Day'].isin(range(2, 5))) & (df['Light'] == 1)][spider_column].mean()

#         if not np.isnan(avg_activities_24hrs_2_4) and not np.isnan(avg_activities_pulse):
#             diff_2_4.append(avg_activities_24hrs_2_4 - avg_activities_pulse)
#             diff_pulse.append(avg_activities_pulse)

#     t_statistic, p_value = stats.ttest_rel(diff_2_4, diff_pulse)
#     return t_statistic, p_value

# # Perform t-test
# t_statistic, p_value = paired_t_test_spiders(merged_df)

# # Save results to CSV
# results_df = pd.DataFrame({'t-statistic': [t_statistic], 'p-value': [p_value]})
# results_df.to_csv('2nd t-Test Steatoda B masking 10pm.csv', index=False)

In [4]:
import pandas as pd
import numpy as np
from scipy import stats

def read_and_preprocess_data(file_path):
    # Read data
    col_names = ["Index", "DateD", "DateM", "DateY", "Time", "MonStatus", "Extras", "MonN", "TubeN", "DataType", "Unused", "Light"]
    for i in range(1, 33):
        col_names.append(f"Sp{i}")
    
    df = pd.read_csv(file_path, names=col_names, sep='\s+', header=None)
    df = df.set_index('Index')

    # Preprocess data
    df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S', errors='coerce')
    df = df[df["MonStatus"] == 1]
    month_map = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6}
    df['DateM'] = df['DateM'].str[:3].map(month_map)
    df['DateY'] = df['DateY'].apply(lambda x: int(str(20) + str(x)))
    df['Date'] = pd.to_datetime(dict(year=df['DateY'], month=df['DateM'], day=df['DateD']), errors='coerce')
    df['Time'] = pd.to_datetime(dict(year=df['Date'].dt.year, month=df['Date'].dt.month, day=df['Date'].dt.day, hour=df['Time'].dt.hour, minute=df['Time'].dt.minute, second=df['Time'].dt.second))
    df = df.drop(["DateD", "DateM", "DateY", "Date", "MonStatus", "Extras", "MonN", "TubeN", "DataType", "Unused"], axis=1)
    day_map = {day: idx+1 for idx, day in enumerate(df['Time'].dt.day.unique())}
    df.insert(0, 'Day', df['Time'].dt.day.map(day_map))
    
    return df

def filter_and_merge(df, threshold=0):
    additional_columns = df.columns[:3]
    filtered_dfs = []

    for day in range(1, 9):  
        day_df = df[df['Day'] == day] 
        count_mov = day_df.filter(like="Sp")  
        x = count_mov.sum(axis=0)  
        z = x > threshold  
        columns_to_keep = z[z].index 
        
        all_columns_to_keep = list(additional_columns) + list(columns_to_keep)
        filtered_df = day_df[all_columns_to_keep]
        filtered_dfs.append(filtered_df)

    merged_df = pd.concat(filtered_dfs)
    merged_df = merged_df.dropna(axis=1)
    
    return merged_df

def paired_t_test_spiders(df):
    diff_2_4 = []
    diff_6_8 = []

    for spider_column in ['Sp1', 'Sp2', 'Sp12', 'Sp14', 'Sp17', 'Sp18', 'Sp19', 'Sp21', 'Sp22', 'Sp23']:  # Selected spider columns
        avg_activities_24hrs_2_4 = df[(df['Day'].isin(range(2, 5)))][spider_column].mean()
        avg_activities_pulse = df[(df['Day'] == 5) & (df['Time'] >= '00:00:00') & (df['Time'] < '02:00:00')][spider_column].sum()
        avg_activities_24hrs_6_8 = df[(df['Day'].isin(range(6, 9)))][spider_column].mean()

        if not np.isnan(avg_activities_24hrs_2_4) and not np.isnan(avg_activities_pulse):
            diff_2_4.append(avg_activities_24hrs_2_4 - avg_activities_pulse)
            diff_6_8.append(avg_activities_24hrs_6_8 - avg_activities_pulse)

    t_statistic, p_value = stats.ttest_rel(diff_2_4, diff_6_8)
    return t_statistic, p_value

# Specify file path
file_path = 'Steatoda A masking 4 am.txt'

# Read and preprocess data
df = read_and_preprocess_data(file_path)

# Filter and merge data
merged_df = filter_and_merge(df)

# Perform t-test
t_statistic, p_value = paired_t_test_spiders(merged_df)

# Save results to CSV
results_df = pd.DataFrame({'t-statistic': [t_statistic], 'p-value': [p_value]})
results_df.to_csv('2nd t-test second trial Steatoda A 4 am.csv', index=False)

print("Results saved to CSV file.")

Results saved to CSV file.
